<font size="1.5">Copyright 2021, by the California Institute of Technology. ALL RIGHTS RESERVED. United States Government sponsorship acknowledged. Any commercial use must be negotiated with the Office of Technology Transfer at the California Institute of Technology.

<font size="1.5">This software may be subject to U.S. export control laws and regulations. By accepting this document, the user agrees to comply with all applicable U.S. export laws and regulations. User has the responsibility to obtain export licenses, or other export authority as may be required, before exporting such information to foreign countries or providing access to foreign persons.


# UAVSAR SLC stack processor

<font size="3">The notebook uses functions from ISCE StripmapStack and some additional functionalities in uavsar_utils folder to create Interferograms from SLC data. 

<font size="3">Currently there is no support to automatically fetch the data. The users are required to copy the download urls from the [UAVSAR webpage](https://uavsar.jpl.nasa.gov/) into a text file locally. 
- If multiple segments exists, download urls for all the segments are needed.
- The notebook reuires ISCE environment. (The last three cells require ISCE + MintPy). 

In [ ]:
import os
import numpy as np
from osgeo import gdal
import glob
import shelve
import isce
import isceobj
import multiprocessing
import subprocess
import sys
import time
from pathlib import Path
n_processes = 4 #int(multiprocessing.cpu_count()) - 2;


### Add the required paths
<font size="3">Add path to the ISCE stripmapStack directory and import functions from the uavsar_utils directory

In [ ]:
#get the path of the ISCE base dir
env_str = os.popen("conda env list | grep '/isce$' | awk '{print $NF}'")
isce_base_dir = env_str.read().strip();
isce2_share_dir = f"{isce_base_dir}/share/isce2";

# now add stripmapStack contrib directory to PATH 
stripmapStack_dir = f"{isce2_share_dir}/stripmapStack";
# add UAVSAR functionalities to the path
uavsar_utils_dir = '/home/jovyan/atbd-se-development/uavsar_utils'
os.environ['PATH'] = f"{uavsar_utils_dir}:{stripmapStack_dir}:{os.environ['PATH']}"
if uavsar_utils_dir not in sys.path:
    sys.path.append(uavsar_utils_dir)
!export OMP_NUM_THREADS=16
try:
    from insar import ImageReader, Interferogram, correlation, cpxlooks, rilooks, isce_xml_ifg
    from unwrap_wlc import unwrap,unwrap_post,unwrap_pre,apply_phasemask,isce_xml_unw
except:
    print('Error: Unble to import key functions. Add uavsar_utils directory to pythonpath')


### Run function to execute commands

In [ ]:
def run_cmd(cmd):
    print("Command: {}".format(cmd))
    try:
        subprocess.run(cmd, shell=True, check=True)
    except subprocess.CalledProcessError as e:
        print("Command failed with error: {}".format(e))

## Place all your inputs here. 
1. Point to your scratch directory or where you would process the data
2. Give a folder name for the processing. 
3. Specify number of looks and number of connections per SLC.
4. Choose a reference data if not the first data will be chosen as reference data
5. A bounding box is required to download the DEM. If not given will be calculated later from the metadata. If there exists an independent DEM file in ISCE readable format, point the path as 'dem_file
6. Custom unwrapping with snaphu configuration parameters is implemented. Requires local installation of snaphu. If not choose 'isce' as unwrapping protocol


In [ ]:
##### In put your Scratch dir
scratch_dir = '/scratch/uavsar'; #define your scratch dir

track_name = 'track_23013'  # Define the UAVSAR directory name
segments = [1, 2]  # Define the SLC segments; Used for concatenation
#reference_date = None # input a reference date if not the first date will be considered reference date

###### Looks and Connections Options #########
rlks = 3  # Define range looks
alks = 12  # Define azimuth looks
numConnections = 2  # Define number of interferogram pairs to be made with each SLC similar to ISCE

print('Processing options\n'
      'range looks={} \n'
      'azimuth looks = {} \n'
      'connections per slc:{} \n'.format(rlks,alks,numConnections));

##### Optional input parameters ########
# reference_date = '20240101'
# bbox = '35 37 -122 -120' # Bounding box for DEM download, if not given will be calculated from the ann file

####### Unwrapping options #############
unwrapping_protocol = 'custom'  # ['isce' or 'custom']
if unwrapping_protocol == 'custom':
    print('Unwrapping will be done using {} options\n'.format(unwrapping_protocol))
    additional_rlks = 3  # default = 3
    additional_alks = 3  # default = 3 
    downsample_interferograms = True  # default= None; If 'True' unwrapping will be done on multilooked interferograms at a lower resolution and the phase cycles obtained will be applied to the 

    snaphu_conf = {
        'DEFAULT_COR_REJECT': 0.4,  # default = 0.4 # Pixels with coherence threshold below will be discarded for unwrapping
        'DEFAULT_COR_SOURCE': 0.45,  # default = 0.45 # phase for pixels from discarded above will be interpolated using nearest pixels with coherence above this
        'DEFAULT_RADIUS': 3.0  # default = 3.0
    }
### custom installatation needs snaphu installed locally
if unwrapping_protocol == 'custom':
    snaphu_path= os.getenv("HOME")+'/'+'tools/bin';
    if os.path.exists(snaphu_path):
        os.environ['PATH'] = f"{snaphu_path}:{os.environ['PATH']}";
    snaphu_test = shutil.which('snaphu');
    if snaphu_test:
        print('Snaphu Installation is works. \n');
        print(snaphu_conf,'\n')
    else:
        print('Locally installed SNAPHU is necessary for implementing custom unwrapping. \n');
        print('Install snaphu or input snaphu bin path. \n'); 
        print('Else choose unwrapping protocol as isce. \n');


### Create processing directory
- Creates a directory as pointed earlier. Rises an error if the scratch directory doesn't exist.

In [ ]:
# Check if the scratch directory exists
if not os.path.exists(scratch_dir):
    print(f'Error: {scratch_dir} does not exist.')
else:
    
    ######## Creating Main directories ###########
    out_dir = os.path.join(scratch_dir, track_name)
    os.makedirs(out_dir, exist_ok=True);
    os.chdir(out_dir)
    print(f'Outputs will be saved to {out_dir}')

    download_dir = os.path.join(out_dir, 'download')
    os.makedirs(download_dir, exist_ok=True)


### Write the download links
- Create a file named download_links.sh in the download folder and copy all your links in it. UAVSAR webpage has 'wget' populated infront of the links already.
- If download is not required, copy the UAVSAR SLCs and *.ann files into the download directory.
- If rerunning is required, move the existing *.slc files from the download directory


In [ ]:
#copy the links from UAVSAR webpage into a file in the download dir named download_links.sh
#this cell shall be updated if a better method of downloading UAVSAR data is available
if any(os.path.exists(os.path.join(out_dir, 'SLC')) or os.path.exists(os.path.join(out_dir, 'SLC_seg'+str(f))) for f in segments):
    print('SLC folder exist. Skipping download');
else:
    print('Downloading UAVSAR SLCs using wget links');
    downloaded_slcs= len(glob.glob(os.path.join(download_dir, '*.slc')));
    if downloaded_slcs<1:
        os.chdir(download_dir);
        !sh download_links.sh > download_log
        !rm *.ann.*
        !rm *.dop.*
        os.chdir(out_dir);
    else:
        print('SLCs exist. Continue to create SLC dir')

### Create SLC directory for  one/multiple SLC segments
- A directory for each segment (Ex:SLC_seg1) is created. If only one segment then SLC directory will be created. 

In [ ]:
#move the dop file to the out dir
dop_file = glob.glob(os.path.join(download_dir, '*.dop'))[0];
if not os.path.exists(dop_file):
    print('Error: *.dop file does not exitst');
#Create SLC directory
if len(segments)>1:
    print('Multiple Segments exist. Creating multiple SLC segments directories!')
    for i in range(len(segments)):
        slc_seg_dir = 'SLC_seg{}'.format(segments[i]);
        if not os.path.exists(slc_seg_dir):
            cmd = 'prepareUAVSAR_coregStack.py -i {} -d {} -o {} -s {}'.format(download_dir,dop_file,slc_seg_dir,segments[i]);
            print(cmd);
            os.system(cmd);
            if i< len(segments)-1:
                cmd = 'cp {}/*/*.ann {}'.format(slc_seg_dir,download_dir);
                print(cmd);
                os.system(cmd);
    slc_dir = os.path.join(out_dir,'SLC_seg{}'.format(segments[0])); 
else:
    print('Procesing only One Segment. No Concatenation')
    slc_dir = os.path.join(out_dir,'SLC');
    cmd = 'prepareUAVSAR_coregStack.py -i {} -d {} -o {} -s {}'.format(download_dir,dop_file,slc_dir,segments[0]);
    os.system(cmd);

print('SLC directory:',slc_dir);


### Give a reference data if not the first data will be considered reference. 

In [ ]:
dateList = sorted(os.listdir(slc_dir));
print('SLC dates:',dateList);
try:
    reference_date
except:
    reference_date=sorted(os.listdir(slc_dir))[0];
print('Reference date for Meta data:',reference_date);
reference_slc_dir = os.path.join(slc_dir,reference_date);

### Read Metadata from the reference SLC

- Here SLC meatadata is loaded into a dictionary variable to query for samples (columns), rows and bounding box for DEM.Each annotation file contains the info for all the segments.

In [ ]:
#read necessary info from the UAVSAR annotation file. Each annotation file contains the info for all the segments.
def read_uavsar_ann_file(ann_file_path):
    '''read UAVSAR ann file as a dict to query for rows, columns info '''
    result_dict = {}
    with open(ann_file_path, 'r') as file:
        for line in file:
            # Strip any leading/trailing whitespace and skip empty lines
            line = line.strip()
            if not line or line.startswith(';'):
                continue
            
            # Split the line into key and value using '=' as the separator
            if '=' in line:
                key, value = line.split('=', 1);
                key = key.split('(')[0].strip()
                value = value.split(';')[0].strip()
                result_dict[key.strip()] = value.strip()
    
    return result_dict
ann_file_path = glob.glob(os.path.join(reference_slc_dir, '*.ann'))[0];
slc_info_dict = read_uavsar_ann_file(ann_file_path);
key = 'slc_{}_1x1 Columns'.format(str(segments[0]))
samples = int(slc_info_dict[key]);
lines = 0;
for seg in segments:
    key = 'slc_{}_1x1 Rows'.format(str(seg))
    lines += int(slc_info_dict[key]);
print('total samples/columns:',samples);
print('total rows:',lines) 
lats= [];lons= [];
try:
    bbox
except:
    for seg in segments:
        for corner in [1,2,3,4]:
            lat, lon = slc_info_dict['Segment {} Data Approximate Corner 1'.format(seg,corner)].split(',')
            lats.append(float(lat));lons.append(float(lon));
    bbox = ' '.join(map(str,[int(np.floor(np.min(lats)-0.1)),int(np.ceil(np.max(lats)+0.1)),int(np.floor(np.min(lons)-0.1)),int(np.ceil(np.max(lons)+0.1))]));
print('Bounding box for DEM:',bbox)

### Update SLC metadata if multiple segments exist
- Total length after concatenating all segments is updated. Coordinates of lower right corner from the last segment is updated
- Changes are made to the metadata of the first SLC segment which will be used subsequently in the other steps

In [ ]:
if len(segments)>1:
    first_seg_dir = os.path.join(out_dir,'SLC_seg{}'.format(segments[0]));
    last_seg_dir = os.path.join(out_dir,'SLC_seg{}'.format(segments[-1]));
    for date in dateList:
        cmd = 'uavsar_update_shelve.py -i {} -d {} -l {} -s2 {}'.format(out_dir,os.path.join(first_seg_dir,date,'data'),lines, os.path.join(last_seg_dir,date,'data'));
        os.system(cmd)

### Download SRTM DEM 

- If using own DEM is preferred, point to it as dem_file.

In [ ]:
#get DEM for geocoding
#if you have your DEM file deifne its path as dem_file '/path/to/dem_file' if not SRTM DEM will be used
#define bounding box for the DEM
try:
    dem_file
except:
    dem_dir = os.path.join(out_dir,'DEM');
    os.makedirs(dem_dir, exist_ok=True);
    try:
        dem_file = glob.glob(os.path.join(dem_dir, "*.dem.wgs84"))[0];
    except:
        os.chdir(dem_dir);
        source_link ="http://step.esa.int/auxdata/dem/SRTMGL1/"
        try:
            bbox
        except:
            print('Bounding box for DEM download required');
        cmd ='dem.py -a stitch -b {} -u {} -d {} -r -s 1 -c'.format(bbox,source_link,dem_dir);
        print(cmd);
        run_cmd(cmd);
        cmd = 'fixImageXml.py -f -i {}'.format(dem_file)
        print(cmd);
        os.system(cmd);
        os.chdir(out_dir);
        dem_file = glob.glob(os.path.join(dem_dir, "*.dem.wgs84"))[0];
print('DEM:{}\n'.format(dem_file))

### Create interferogram pairs list
- Based on the numConnections variable input earlier. 

In [ ]:
#create pairs
looks = [alks, rlks];
ifg_dir =  os.path.join(out_dir,'Igrams');
print('Selecting pairs with {} connections per SLC \n'.format(numConnections));
os.makedirs(ifg_dir, exist_ok=True);
pairs_list= [];
for ii,dd in enumerate(dateList):
    referenceDate = dd;
    for jj in range(ii+1, ii+1+numConnections):
        if jj < len(dateList):
            secondaryDate = dateList[jj];
            ifg_pair =  referenceDate+'_'+secondaryDate;
            pairs_list.append(ifg_pair);
print('Pairs:','\n')
print(pairs_list,'\n');

### Generate Interferometic products
- The code multilooks the SLCs and creates interferograms. Processes multiple segments simultenously. 

In [ ]:
print('Genrating Interferogram, Amplitude and coherence files \n');
print('Interferogram is concatenated from multiple SLC segments');

def cat_and_interfere(slc_dir,samples,looks,segments,ifg_dir,pair):
    referenceDate,secondaryDate = pair.split('_');
    int_dir  = os.path.join(ifg_dir,referenceDate+'_'+secondaryDate);os.makedirs(int_dir, exist_ok=True);
    intf =  os.path.join(int_dir,referenceDate+'_'+secondaryDate+'.int');
    ampf =  os.path.join(int_dir,referenceDate+'_'+secondaryDate+'.amp');
    cohf =  os.path.join(int_dir,referenceDate+'_'+secondaryDate+'.coh');
    igram = Interferogram(samples, looks);
    int_samples = samples//looks[1];
    if not all(os.path.exists(f) for f in (intf, ampf, cohf)):
        if len(segments)>1:
            with open(intf, 'wb') as fint, open(ampf, 'wb') as famp:
                for seg in segments:
                    ref_img = slc_dir[:-1]+str(seg)+'/'+referenceDate+'/'+referenceDate+'.slc';#print(ref_img)
                    sec_img = slc_dir[:-1]+str(seg)+'/'+secondaryDate+'/'+secondaryDate+'.slc';#print(sec_img)
                    img0 = ImageReader(ref_img, samples, dtype='complex64')
                    img1 = ImageReader(sec_img, samples, dtype='complex64')
                    #log.info('... %s * conj(%s)', img0.filename, img1.filename)
                    for int_row, amp_row in igram.iterrows(img0, img1):
                        int_row.tofile(fint);
                        amp_row.tofile(famp);
            correlation(intf, ampf, cohf);
            isce_xml_ifg(intf, ampf, cohf, int_samples)
        else:
            ref_img = slc_dir+'/'+referenceDate+'/'+referenceDate+'.slc';#print(ref_img)
            sec_img = slc_dir+'/'+secondaryDate+'/'+secondaryDate+'.slc';#print(sec_img)
            with open(intf, 'wb') as fint, open(ampf, 'wb') as famp:
                img0 = ImageReader(ref_img, n, dtype='complex64')
                img1 = ImageReader(sec_img, n, dtype='complex64')
                #log.info('... %s * conj(%s)', img0.filename, img1.filename)
                for int_row, amp_row in igram.iterrows(img0, img1):
                        int_row.tofile(fint);
                        amp_row.tofile(famp);
            correlation(intf, ampf, cohf);
            isce_xml_ifg(intf, ampf, cohf, int_samples)
    else:
        print('Skipping {} as files exist \n'.format(os.path.basename(intf)));
with multiprocessing.Pool(processes=12) as pool:
    pool.starmap(cat_and_interfere,[(slc_dir,samples,looks,segments,ifg_dir,pair) for pair in pairs_list]);
print('Finished generating Inteferograms');

### Unwrap the interferograms using Custom option
1. The process requires snaphu
2. The process replaces phase on low coherence pixels with interpolated phase from nearest high coherence pixels
3. Performs unwrapping initially on a low resoution interferogram and adds the corresponding number of phase cycles to the high resolution interferogram. Enhances the processing time and performance of unwrapping.

In [ ]:
#unwrap interferograms

if unwrapping_protocol == 'custom':
    import json
    ifg_info = {
      "looks_azimuth": alks,
      "looks_range": rlks,
      "samples_int": samples//rlks,
      "samples_slc": samples
    }
    int_json_file = os.path.join(ifg_dir,'int.json');
    with open(int_json_file,'w') as json_file:
        json.dump(ifg_info,json_file,indent=4)
    for pair in pairs_list:
        shutil.copy(int_json_file,os.path.join(ifg_dir,pair))
        
    ### function to downsamsaple interferograms
    def downsample_int(hires_dir,lores_dir,nr,na,pair):
        dir_in = os.path.join(hires_dir, pair);
        dir_out = os.path.join(lores_dir, pair);
        if dir_in == dir_out:
            raise ValueError('Input and output directories must differ \n')

        dir_out = os.path.abspath(dir_out)
        os.makedirs(dir_out, exist_ok=True);
        # Load the JSON file describing the interferograms and update it with the
        # new number of looks.
        input_json_file= os.path.join(dir_in,'int.json');
        output_json_file= os.path.join(dir_out,'int.json');
        if not os.path.exists(input_json_file):
            raise ValueError('Missing int.json file in the high resolution interferogram directory \n')

        with open(input_json_file) as f:
            info = json.load(f)

        n = info['samples_int']
        info['samples_int'] = n // nr
        info['looks_range'] *= nr
        info['looks_azimuth'] *= na

        with open(output_json_file, 'w') as f:
            json.dump(info, f, indent=2, sort_keys=True)

        looks = (na, nr)

        intf_in = glob.glob(os.path.join(dir_in,'*.int'))[0];
        amp_in = intf_in[:-4] + '.amp';
        if not os.path.exists(amp_in):
            raise IOError('Found int=%s but no matching .amp file.' % intf_in)
        intf_out = os.path.join(dir_out, os.path.basename(intf_in))
        amp_out = os.path.join(dir_out, os.path.basename(amp_in))
        cor_out = amp_out[:-4] + '.coh';
        if not all(os.path.exists(f) for f in (intf_out, amp_out, cor_out)):
            cpxlooks(intf_in, intf_out, n, looks)
            rilooks(amp_in, amp_out, n, looks)
            correlation(intf_out, amp_out, cor_out)
        else:
            print('Skipping downsampling of {}. Downsampled files already exist \n'.format(os.path.basename(intf_in)))
    
    ### function to run wrapping on a downsampled interferograms and phase ambiguity solution to original interferograms

    def unwrap_high_low(lores_ifg_dir,hires_ifg_dir,pair,snaphu_conf,clean=False):
        dir_hi = os.path.join(hires_ifg_dir, pair);
        dir_low = os.path.join(lores_ifg_dir, pair);
        cor_reject = snaphu_conf['DEFAULT_COR_REJECT']
        cor_source = snaphu_conf['DEFAULT_COR_SOURCE']
        radius = snaphu_conf['DEFAULT_RADIUS']
        if dir_hi is None:
            dir_hi = dir_low

        for intf in glob.glob(os.path.join(dir_hi, '*.int')):
            # If --highres=dir_hi option is given, assume that the high-resolution
            # products are the ones of interest (e.g., still skip if low-res unw
            # does not exist).
            unw = intf[:-4] + '.unw'
            if os.path.exists(unw):
                print('Skipping because {} it already exists.'.format(os.path.basename(unw)));
            else:
                # Make sure necessary input files exist.
                intf_low = os.path.join(dir_low, os.path.split(intf)[1])
                cor_low = intf_low[:-4] + '.coh' # TO modif
                if os.path.isfile(cor_low) == False:
                    cor_low = intf_low[:-4] + '.cor'
                info_low = os.path.join(dir_low, 'int.json')
                for f in (intf_low, cor_low, info_low):
                    if not os.path.exists(f):
                        raise IOError('Could not find %s needed to make %s.' % (f, unw))
                # checking of the low res product already exists, if yes skip it as it could be phase corrected
                unw_low = intf_low.replace('.int', '.unw')      # DB: start 29 June 2016
                if os.path.exists(unw_low):
                    log.debug('Skipping %s because it already exists.\n', unw_low)
                    pre_int = intf_low + '.pre'
                    pre_info = intf_low + '.pre.h5'
                else:
                    # Inpaint low correlation areas of the interferogram.
                    with open(info_low) as f:
                        n = json.load(f)['samples_int']
                    pre_int = intf_low + '.pre'
                    pre_info = intf_low + '.pre.h5'
                    unwrap_pre(intf_low, cor_low, n, pre_int, pre_info,
                               cor_reject=snaphu_conf['DEFAULT_COR_REJECT'], cor_source=snaphu_conf['DEFAULT_COR_SOURCE'], 
                               sigma=snaphu_conf['DEFAULT_RADIUS']);
                    # Unwrap the simplified interferogram with SNAPHU.
                    pre_unw = pre_int + '.unw'
        #            unwrap(pre_int, pre_unw)   # original version of the code 
                    unwrap(pre_int, pre_unw, cor_low)   # Use this command if you want snaphu to account for real coherence
                    # Put the original data back in on a reasonable phase ambiguity.
                    unw_low = intf_low.replace('.int', '.unw')
                    unwrap_post(pre_unw, pre_info, unw_low)     # DB: end 

                # Apply the low resolution phase ambiguity solution to high res data.
                if unw != unw_low:
                    apply_phasemask(unw_low, intf, unw)
                if clean==True:
                    for f in (pre_int, pre_info):
                        if os.path.exists(f):        #DB: start 29 June 2016
                            os.remove(f)             #DB: en

    def generate_xml_for_unw(ifg_dir,pair,slc_dir,looks):
        unwrapFile = "{b}/{a}/{a}.unw".format(a=pair, b=ifg_dir);
        int_file = "{b}/{a}/{a}.int".format(a=pair, b=ifg_dir)
        unw_xml_file = unwrapFile+'.xml';
        if not os.path.exists(unw_xml_file):
            ## Amplitude layer, taken from the interferogram
            ds = gdal.Open(int_file, gdal.GA_ReadOnly)
            igram = ds.GetRasterBand(1).ReadAsArray();
            length, width = np.shape(igram)
            #print ('width, length =', width, length)
            ds = None;
            if os.path.getsize(unwrapFile) != os.path.getsize(int_file):    
                amp = np.abs(igram) # Compute amplitude from ifg

                ## Generate 2 band unw product by adding amp file to unwrapped ifgram
                # add amp file to unwrapped ifgram
                f1 = unwrapFile
                X1 = np.memmap(f1, dtype=np.float32)
                igram_unw = X1.reshape((length,width))
                unw_pha = igram_unw

                ## Generate 2 band unw product
                new_unw = unwrapFile
                unw_data = np.array(np.hstack((amp, unw_pha)), dtype=np.float32)
                unw_data.tofile(new_unw);
            
            ## generate referenceShelve dir
            referenceDate,secondaryDate = pair.split('_');
            reference_slc_dir = os.path.join(slc_dir,referenceDate);
            interferogramDir = os.path.dirname(int_file)
            referenceShelveDir = os.path.join(interferogramDir , 'referenceShelve')
            os.makedirs(referenceShelveDir, exist_ok=True);
            cpCmd='cp ' + os.path.join(reference_slc_dir, 'data*') +' '+referenceShelveDir
            os.system(cpCmd)
            pckfile = os.path.join(referenceShelveDir,'data');

            ## generate xml file for unw
            isce_xml_unw(unwrapFile, width,length,2)
            
        else:
            print('Skipping {} as xml file exists'.format(os.path.basename(unwrapFile)));
    
    if downsample_interferograms == True:
        lowres_ifg_dir= os.path.join(out_dir,'Igrams3'+'_'+str(rlks*additional_rlks)+'_'+str(alks*additional_alks));
        os.makedirs(lowres_ifg_dir, exist_ok=True);
        with multiprocessing.Pool(processes=12) as pool:
            pool.starmap(downsample_int,[(ifg_dir,lowres_ifg_dir,additional_rlks,additional_alks,pair) for pair in pairs_list]);
    else:
        lowres_ifg_dir= ifg_dir;
    for pair in pairs_list:
        unwrap_high_low(lowres_ifg_dir,ifg_dir,pair,snaphu_conf,True);
        generate_xml_for_unw(ifg_dir,pair,slc_dir,looks);

    # with multiprocessing.Pool(processes=12) as pool:
    #     pool.starmap(unwrap_high_low,[(ifg_dir,lowres_ifg_dir,pair,snaphu_conf,True) for pair in pairs_list]);
    print('Finished Unwrapping');
else:
    print('Proceed to Unwrap using ISCE unwrap.py')

### Unwrap the interferograms using ISCE+SNAPHU
- If custom unwrapping is not desired, proceeds with standard unwrapping procedure from ISCE 2.0

In [ ]:
#unwrap interferograms
if unwrapping_protocol == 'isce':
    def unwrap_snaphu(unwrap_method,defo_max,looks,slc_dir,ifg_dir,pair):
        referenceDate,secondaryDate = pair.split('_');
        reference_slc_dir = slc_dir+'/'+referenceDate
        int_dir  = os.path.join(ifg_dir,referenceDate+'_'+secondaryDate);
        intf =  os.path.join(int_dir,referenceDate+'_'+secondaryDate+'.int');
        unwf =  os.path.join(int_dir,referenceDate+'_'+secondaryDate+'.unw');
        cohf =  os.path.join(int_dir,referenceDate+'_'+secondaryDate+'.coh');
        if not os.path.exists(unwf):
            cmd = ['unwrap.py -i {} -c {} -u {} -s {}/ -a {} -r {} -d {} -m {}'.format(intf,cohf,unwf,reference_slc_dir,looks[0],looks[1],defo_max,unwrap_method)];
            run_cmd(cmd);
        else:
            print('Skipping {} as unw file exists'.format(os.path.basename(unwf)));
    
    unwrap_method = 'snaphu';
    defo_max = 2;
    for pair in pairs_list:
        unwrap_snaphu(unwrap_method, defo_max, looks, slc_dir, ifg_dir, pair);
    print('Finished Unwrapping')
    
#####Currently not working, crashing the instance ########
# with multiprocessing.Pool(processes=12) as pool:
#     pool.starmap(unwrap_snaphu,[(unwrap_method,defo_max,looks,slc_dir,ifg_dir,pair) for pair in pairs_list]);
# if __name__ == "__main__":
#     with ProcessPoolExecutor(max_workers=12) as executor:
#         executor.map(unwrap_snaphu, pairs_list)

### Generate baselines

In [ ]:
#create UAVSAR baselines
baselines_dir = os.path.join(out_dir,'baselines');
os.makedirs(baselines_dir, exist_ok=True);
if len(glob.glob(os.path.join(baselines_dir,'*.txt')))== (len(os.listdir(slc_dir))-1):
    print('Baseline directory and files exist. Skipping')

else:
    cmd ='uavsar_baselines.py -s {} -b {}  -m {}'.format(slc_dir,baselines_dir,reference_slc_dir);
    run_cmd(cmd);


### Generate geometry files from the DEM

In [ ]:
#generate geometry files
geometery_dir = os.path.join(out_dir,'geometry');
if os.path.exists(os.path.join(geometery_dir,'hgt.rdr')):
    print('Geometry files exist. Skipped running topo.py');
else:
    cmd = 'topo.py -a {} -r {} -d {} -m {} -o {} -n'.format(str(alks), str(rlks),dem_file,reference_slc_dir,geometery_dir);
    print(cmd);
    run_cmd(cmd);
    geometery_dir_mlooked = os.path.join(os.path.dirname(os.path.dirname(geometery_dir)), 'geom_reference');
    cmd = 'mv {}/* {}/.'.format(geometery_dir_mlooked,geometery_dir);
    print(cmd);
    run_cmd(cmd)

### Copy referenceShelve directory from the master interferogram

In [ ]:
#create referenceShelve dir
reference_dir = os.path.join(out_dir,'referenceShelve');
reference_ifg = sorted(glob.glob(ifg_dir+'/'+reference_date+'_*'))[0];
print('Reference interferogram is:',reference_ifg)
cmd ='cp -r {}/referenceShelve {}'.format(reference_ifg,reference_dir)
print(cmd);
run_cmd(cmd);

### Prepare interferograms and geometry files to be loaded into MintPy.
- This step requires an environment with both ISCE and MintPy.

In [ ]:
#Prepare .rsc files for unwrapped interferograms to be loaded into MintPy
cmd = 'prep_isce.py -f {}/*/{} -m {}/data -b {}  -g {}'.format(ifg_dir,'*.unw',reference_dir,baselines_dir,geometery_dir)
print(cmd);
run_cmd(cmd);

### Add paths pointing the inteferometric and geometry files to the smallbaselineApp.cfg file

In [ ]:

mintpy_dir= os.path.join(out_dir,'mintpy');
os.makedirs(mintpy_dir, exist_ok=True);
config_file = Path(mintpy_dir)/('smallbaselineApp.cfg')
config_file_parameters = """
####mintpy parameters
mintpy.load.processor = isce
mintpy.compute.numWorker = auto
##path to interferograms
mintpy.load.autoPath = no
mintpy.load.metaFile       = {out_dir}/referenceShelve/data.dat
mintpy.load.baselineDir    = {out_dir}/baselines  
mintpy.load.unwFile        = {out_dir}/Igrams/*/*.unw  
mintpy.load.corFile        = {out_dir}/Igrams/*/*.coh
mintpy.load.connCompFile   = {out_dir}/Igrams/*/*.conncomp  
#mintpy.load.intFile        = {out_dir}/Igrams/*/*.int  
###path to geometry files
mintpy.load.demFile = {geometery_dir}/hgt.rdr
mintpy.load.lookupYFile = {geometery_dir}/lat.rdr
mintpy.load.lookupXFile = {geometery_dir}/lon.rdr
mintpy.load.incAngleFile = {geometery_dir}/los.rdr
mintpy.load.azAngleFile = {geometery_dir}/los.rdr
mintpy.load.shadowMaskFile = {geometery_dir}/shadowMask.msk
""".format(out_dir=out_dir,geometery_dir=geometery_dir)
config_file.write_text(config_file_parameters);
print('MintPy config file:\n    {}:'.format(config_file))
print(config_file.read_text())

### Load files to make MintPy input data cube

In [ ]:
os.chdir(mintpy_dir);
cmd = 'smallbaselineApp.py ' + 'smallbaselineApp.cfg' + ' --dostep load_data'
print(cmd);
run_cmd(cmd);

### Process Timeseries inversion and other Mintpy steps (optional)
- Proceed only if required

In [ ]:
#optional continue if you like to do mintpy processing here
cmd = 'smallbaselineApp.py ' + 'smallbaselineApp.cfg' + ' --start modify_network'
print(cmd);
run_cmd(cmd);